# PROJET ANALYSE DE DONNEES 

In [ ]:
library(ggforce)
library(ggplot2)
library(reshape2)
library(plotly)
library(corrplot)
library(FactoMineR)
library(factoextra)
library(gridExtra)
library(cluster)
library(dendextend)
library(dplyr)
library(mclust)

In [ ]:
loading <- read.csv("data/velibLoading.csv", sep = " ")
head(loading)

In [ ]:
Coord <- read.csv("data/velibCoord.csv", sep = " ")
head(Coord)

On cherche maintenant à voir si les deux tableaux contiennent des données manquantes.

In [ ]:
print('--Loading--')
any(is.null(loading)) 
#Nous n'avons pas de données manquantes dans les colonnes de loading 
print('--Coord--')
any(is.null(Coord))
#pareil pour coord

On cherche également à voir si certaines données sont dupliquées dans les deux df.

In [ ]:
print('--Loading--')
any(duplicated(loading)) 
#Nous n'avons pas de données dupliquées dans les colonnes de loading 
print('--Coord--')
any(duplicated(Coord))
#pareil pour coord

# Analyse Descriptive

In [ ]:
# Changement des options graphiques
options(repr.plot.width = 10, repr.plot.height = 10, repr.plot.res = 100)
#On affiche le chargement de la 1ère station

#On créé une séquence de temps
p <- ncol(loading)
Time <- seq(1, p)

#On garde seulement la 1ère station
loading_transposed <- t(loading)
first_column <- loading_transposed[, 1]

plot(Time, first_column, type = "l", lwd = 2, col = "blue", xlab = "Time", ylab = "Loading", main =Coord$names[1])
abline(v = seq(1, p, length.out = 8), col = "black", lty = "dotted")

On va désormais parcourir les 16 premières stations pour afficher leur chargement.

In [ ]:
par(mfrow = c(4, 4), mar = c(4, 4, 2, 1)+ 0.1, oma = c(0, 0, 4, 0)+ 0.1, mgp = c(3.5, 1.5, 0))

p <- ncol(loading)
Time <- 1:p
 
for (i in 0:3) {
  for (j in 0:3) {
    id_station <- 4 * i + j + 1
    plot(Time, loading_transposed[, id_station], type = "l", col = "blue", lwd = 2, xlab = "Time", ylab = "Loading",
         main = Coord$names[id_station])
    abline(v = seq(1, p, length.out = 8), col = "black", lty = "dotted")
  }
}

Boxplot des chargements pour chaque heure.

In [ ]:

bp <- boxplot(as.matrix(loading), 
              col = "white", border = "black", median.col = "red",
              staplewex = 0, notch = FALSE, outline = FALSE,
              names = rep("", ncol(loading)))

abline(v = seq(1, ncol(loading), length.out = 8), col = "purple",lwd=4)


title <- "Boxplots"
ticks <- seq(0, 168, by = 5)
labels <- seq(0, 168, by = 5)
title(main = title, cex.main = 1.25)
axis(1, at = ticks, labels = labels, cex.axis = 1.25)
mtext("Time", side = 1, line = 2, cex = 1.5)
mtext("Loading", side = 2, line = 2, cex = 1.5)

On voit que les jours de la semaine ont des comportements similaires entre eux. On voit aussi que les deux jours de week-end se ressemblent. <br>
<br>
## Moyenne de chargement par heure pour chaque jour de la semaine

In [ ]:
MoyHeures <- colSums(loading) / 1189


jours <- c("Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi", "Samedi", "Dimanche")

MoyHeuresPJours <- matrix(MoyHeures, nrow = 24)
MoyHeuresPJours <- t(aperm(MoyHeuresPJours, c(2, 1)))

plot(1:24, MoyHeuresPJours[,1], type = "l", xlab = "Heures", ylab = "Loading", 
     col = rainbow(7)[1], ylim = range(MoyHeuresPJours), 
     main = "Moyenne de chargement par heure pour chaque jour de la semaine")

for (i in 2:7) {
  lines(1:24, MoyHeuresPJours[,i], type = "l", col = rainbow(7)[i])
}
legend("topright", legend = jours, col = rainbow(7), lwd = 2, cex =0.8, bty = "n")

On voit des résultats similaires pour samedi et dimanche, qui ont une tendance différente des autres jours de la semaine, ce qui confirme ce qui est vu sur le boxplot.

## Représentation des moyennes de chargement pour chaque heure (6h,12h,23h)

In [ ]:
#On garde les coordonnées géographiques de Paris pour zoomer direct sur la ville
paris_lon <- 2.3522
paris_lat <- 48.8566

#On garde les positions des stations
Position <- Coord[, c(1, 2)]

#On garde les heures à afficher
a6 <- seq(6, 168, 24)
a12 <- seq(12, 168, 24)
a23 <- seq(23, 168, 24)

#On calcule les moyennes de chargement pour chaque heure


#----------------6h---------------------------------



data6h <- rowMeans(loading[, a6])
data6h_df <- Coord 
data6h_df$loading_six_heures=data6h
fig6 <-plot_ly(data = data6h_df, x = ~longitude, y = ~latitude, type = "scattermapbox",
                    color = ~loading_six_heures, colors = c("blue", "gold"),
                    marker = list(size = 7), zoom = 10, text = paste("Loading:", round(data6h, 2)),
                    hoverinfo = "text") %>%
            layout(mapbox = list(center = list(lon = paris_lon, lat = paris_lat),
                                 zoom = 10, style = "carto-positron"))
fig6


#----------------12h---------------------------------

data12h <- rowMeans(loading[, a12])
data12h_df <- Coord
data12h_df$loading_douze_heures=data12h

fig12 <-plot_ly(data = data12h_df, x = ~longitude, y = ~latitude, type = "scattermapbox",
                    color = ~loading_douze_heures, colors = c("blue", "gold"),
                    marker = list(size = 7), zoom = 10, text = paste("Loading:", round(data12h, 2)),
                    hoverinfo = "text") %>%
            layout(mapbox = list(center = list(lon = paris_lon, lat = paris_lat),
                                 zoom = 10, style = "carto-positron"))
fig12

#----------------23h---------------------------------


data23h <- rowMeans(loading[, a23])
data23h_df <- Coord
data23h_df$loading_vingt_trois_heures=data23h
fig23 <- plot_ly(data = data23h_df, x = ~longitude, y = ~latitude, type = "scattermapbox",
                    color = ~loading_vingt_trois_heures, colors = c("blue", "gold"),
                    marker = list(size = 7), text = paste("Loading:", round(data23h, 2)),
                    hoverinfo = "text") %>%
            layout(mapbox = list(center = list(lon = paris_lon, lat = paris_lat),
                                 zoom = 10, style = "carto-positron"))
fig23

## Représentation des stations en fonction de leur relief

In [ ]:
#On calcule la moyenne des données de chargement pour chaque ligne

data24h <- rowMeans(loading)


figbonus <- plot_ly(data = Coord, x = ~longitude, y = ~latitude, type = "scattermapbox",
                    color = ~as.factor(bonus), colors = c("cornflowerblue", "gold"),
                    marker = list(size = 7), text = paste("Loading:", round(data24h, 2)),
                    hoverinfo = "text") %>%
            layout(mapbox = list(center = list(lon = paris_lon, lat = paris_lat),
                                 zoom = 10, style = "carto-positron"))

figbonus

On reconnaît ici les collines parisiennes (représentées en jaune sur la carte).

## Histogramme des données de chargement

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10, repr.plot.res = 100)
hist(unlist(loading), breaks = 64, main = "Histogramme des données de chargement", 
     xlab = "Valeurs de chargement", ylab = "Fréquence")

On a beaucoup de stations très peu chargées (à une heure de donnée), difficile à interpréter.

Rq: On a 1189 stations pour 168h de prélèvement de chargement donc cohérent d'avoir un nombre de 25000 en max.

## Matrice de corrélation 

In [ ]:
correlation <- cor(loading)

for (k in 0:6)
  {correlation_subset <- correlation[(1+24*k):(24*(k+1)), (1+24*k):(24*(k+1))]
  corrplot(correlation_subset, method = "ellipse", type = "upper")}

On affiche les corrélations des heures pour chaque jour de la semaine et du week-end.

In [ ]:
selected_columns <- c("Lun.12", "Lun.13", "Mar.12", "Mar.13", "Ven.12", "Ven.13", "Lun.18", "Ven.18")

pairs(loading[selected_columns], 
      pch = 20,               
      col = "skyblue",           
      cex = 0.3,              
      gap = 0.2)

Ce graphique confirme les corrélations que l'on a pu voir sur le graphe au dessus. Par exemple Lundi-12h et Lundi 13h sont très corrélés (on observe une droite). Pour mardi et vendredi il y a aussi une corrélation entre 12 et 13h.

## Comportements stations sur colline ou non

### Sur colline : 

In [ ]:
#---------------Boxplot----------------

loading_hill = loading[Coord$bonus==1,]
  
bp <- boxplot(as.matrix(loading_hill), 
              col = "white", border = "black", median.col = "red",
              staplewex = 0, notch = FALSE, outline = FALSE,
              names = rep("", ncol(loading_hill)))

abline(v = seq(1, ncol(loading_hill), length.out = 8), col = "purple",lwd=4)


title <- "Boxplots sur colline "
ticks <- seq(0, 168, by = 5)
labels <- seq(0, 168, by = 5)
title(main = title, cex.main = 1.25)
axis(1, at = ticks, labels = labels, cex.axis = 1.25)
mtext("Time", side = 1, line = 2, cex = 1.5)
mtext("Loading", side = 2, line = 2, cex = 1.5)

#Comportements moyen
MoyHeures <- colSums(loading_hill) / nrow(loading_hill)


jours <- c("Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi", "Samedi", "Dimanche")

MoyHeuresPJours <- matrix(MoyHeures, nrow = 24)
MoyHeuresPJours <- t(aperm(MoyHeuresPJours, c(2, 1)))

plot(1:24, MoyHeuresPJours[,1], type = "l", xlab = "Heures", ylab = "Loading", 
     col = rainbow(7)[1], ylim = range(MoyHeuresPJours), 
     main = "Moyenne de chargement sur colline par heure pour chaque jour de la semaine")

for (i in 2:7) {
  lines(1:24, MoyHeuresPJours[,i], type = "l", col = rainbow(7)[i])
}
legend("topright", legend = jours, col = rainbow(7), lwd = 2, cex =0.8, bty = "n")


Pour les stations en hauteur, les week-ends il n'y a pas beaucoup de vélos chargés. En semaine, il y a un fort écart entre le matin et l'après-midi.

### Sur stations sans relief : 

In [ ]:
#---------------Boxplot----------------

loading_no_hill = loading[Coord$bonus==0,]
  
bp <- boxplot(as.matrix(loading_no_hill), 
              col = "white", border = "black", median.col = "red",
              staplewex = 0, notch = FALSE, outline = FALSE,
              names = rep("", ncol(loading_no_hill)))

abline(v = seq(1, ncol(loading_no_hill), length.out = 8), col = "purple",lwd=4)


title <- "Boxplots pas sur colline"
ticks <- seq(0, 168, by = 5)
labels <- seq(0, 168, by = 5)
title(main = title, cex.main = 1.25)
axis(1, at = ticks, labels = labels, cex.axis = 1.25)
mtext("Time", side = 1, line = 2, cex = 1.5)
mtext("Loading", side = 2, line = 2, cex = 1.5)

#Comportements moyen
MoyHeures <- colSums(loading_no_hill) / nrow(loading_no_hill)


jours <- c("Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi", "Samedi", "Dimanche")

MoyHeuresPJours <- matrix(MoyHeures, nrow = 24)
MoyHeuresPJours <- t(aperm(MoyHeuresPJours, c(2, 1)))

plot(1:24, MoyHeuresPJours[,1], type = "l", xlab = "Heures", ylab = "Loading", 
     col = rainbow(7)[1], ylim = range(MoyHeuresPJours), 
     main = "Moyenne de chargement pas sur colline par heure pour chaque jour de la semaine")

for (i in 2:7) {
  lines(1:24, MoyHeuresPJours[,i], type = "l", col = rainbow(7)[i])
}
legend("topright", legend = jours, col = rainbow(7), lwd = 2, cex =0.8, bty = "n")

On voit que le comportement pour les stations plates se rapproche plus du comportement moyen, ce qui est normal car la majorité des stations ne sont pas sur des collines. 
De plus on remarque une différence moins flagrante entre la semaine et le weekend pour les stations plates.


# PCA

On a un jeu de données pour loading a 168 variables donc 168 dimensions. On veut essayer de voir si on peut se ramener à des données de plus faibles dimensions. Pour cela on va faire une Analyse en Composantes Principales.

## Implémentation de la PCA

In [ ]:
#On normalise les données

loading_scaled <- scale(loading)

#Analyse en composantes principales (ACP)
pca <- prcomp(loading_scaled)

#Pour l'affichage on garde les 25 premières dimensions

nb_components=25

#Variance expliquée par composante principale
barplot(pca$sdev^2 / sum(pca$sdev^2), 
        main = "Variance expliquée par composante principale",
        xlab = "Composante principale",
        ylab = "Variance expliquée",
        ylim = c(0, 1),
        xlim = c(1,25))

#Variance expliquée cumulée par composante principale
barplot(cumsum(pca$sdev^2 / sum(pca$sdev^2)), 
        main = "Variance expliquée cumulée par composante principale",
        xlab = "Composante principale",
        ylab = "Variance expliquée cumulée",
        ylim = c(0, 1),
        xlim = c(1,25))

#On trace la limite de 70% de variance expliquée
abline(h = 0.7, col = "red")

Pour expliquer au moins 70% de variance, il faut garder 4 composantes principales.

### Boxplots des projections des données sur les composantes principales

In [ ]:
C <- as.data.frame(pca$x[, 1:nb_components])

boxplot(C, main = "Boxplot des projections des données sur les composantes principales",
        xlab = "Composante principale", ylab = "Valeur", col = "skyblue")

Ce graphique montre une grande variance pour les 3/4 premières composantes principales, ce qui confirme le choix de garder 4 composantes.
On voit par exemple que les individus ont en moyenne -1 pour la coordonnée sur la 1ère composante principale.


## Projection des variables sur le plan d'ACP

In [ ]:
#Selon la 1ère et la 2ème composantes principales

options(repr.plot.width = 10, repr.plot.height = 8)

grid.arrange(
    fviz_eig(pca), 
    fviz_pca_var(pca,axes=c(1,2)),
    ncol=2
)

On voit que pour les dimensions 1 et 2, toutes les flèches sont proches du cercle : les variables sont donc relativement bien expliquées par les composantes principales.

In [ ]:
#Selon la 1ère et la 3ème composantes principales
options(repr.plot.width = 10, repr.plot.height = 8)

grid.arrange(
    fviz_eig(pca), 
    fviz_pca_var(pca,axes=c(1,3),label='none'),
    ncol=2
)

On voit que pour les dimensions 1 et 3 les flèches sont plus courtes : les variables sont moins bien expliquées (ce qui est normal car la dimension 3 explique moins de variance que la dimension 2).

## Qualité et contribution des variables

In [ ]:
#On affiche la qualité de représentation des variables : comment les axes permettent d'expliquer les variables
q1=fviz_pca_var(pca, col.var="cos2",repel=TRUE,gradient.cols=c("#00afbb","red","yellow"),label="none")
#On affiche la contribution des variables aux axes
c1=fviz_pca_var(pca,col.var="contrib",repel=TRUE,gradient.cols=c("#00afbb","red","yellow"),label="none")
grid.arrange(q1,c1)

## Interprétation des dimensions de l'ACP

Dans cette partie, nous rappelons que les composantes principales sont des combinaisons linéaires des variables initiales.

In [ ]:
par(mfrow = c(4, 2), mar = c(4, 4, 2, 1))

#On utilise les 168h en x
u <- seq(0, 168, length.out = 168)
for (i in 1:4) {
  plot(u, pca$rotation[, i], type = "l", ylim = c(-0.2, 0.2), xlab = "Longueur d'onde", ylab = "", main = paste("Composante", i))
  abline(h = mean(pca$rotation[,i]), col = "red",lty=2) 
}

Les 4 plots représentent les coefficients pour les 168 heures, et pour les 4 premières dimensions de l'ACP.

Le premier plot (sur la dimension 1) on voit que tous les coeffs sont à peu près égaux, ce qui montre que la 1ère composantes est proportionnelle à la moyenne des chargements sur toutes les stations au cours des heures.

Le deuxième plot (sur la dimension 2) montre le contraste entre le jour et la nuit: en pleine journée les coeffs sont positifs alors que la nuit ils sont négatifs.

Les plots 3 et 4 sont plus difficiles à interpréter, même si on remarque plutôt sur le troisième plot (dimension 3) le contraste entre les jours en pleine semaine et le week-end.


Ce résultat se retrouve sur le cercle des corrélations : sur la composante 1, les variables ont à peu près la même coordonnées (même position sur l'axe x), tandis que sur la composante 2, il y a 2 groupes qui semblent se dégager. En haut on distingue les heures correspondant à la journée, et en bas à la nuit (les heures entre ces deux périodes : début de matinée ou fin de soirée se retrouvent avec une coordonnée proche de 0 selon l'axe y).

## Projection des individus sur le plan ACP

In [ ]:
fviz_pca_ind(pca,geom=c("point"))

## Qualité et contribution des individus

In [ ]:
#On affiche la qualité de représentation et la contribution des individus
q2=fviz_pca_ind(pca, col.ind="cos2",geom=c("point"),gradient.cols=c("#00afbb","red","yellow"))
c2=fviz_pca_ind(pca, col.ind="contrib",geom=c("point"),gradient.cols=c("#00afbb","red","yellow"))
grid.arrange(q2,c2)

On voit que les individus proches du centre (origine) sont mal expliqués par les composantes principales. 
Pour la contribution on voit la même chose pour les individus du centre qui contribuent moins aux composantes principales.


# Clustering

Nous allons maintenant essayer de regrouper les stations en différentes classes. Pour cela utiliser des méthodes de Clustering.
Commenons d'abord par du clustering Kmeans.

# Kmeans

In [ ]:
velib_PCA_reduced= C[,1:4]  

## Choix du nombre de clusters 

### Avec l'inertie : 

In [ ]:
K=10
Iintra=NULL
for (k in 2:K)
{
  #On calcule l'inertie pour différentes valeurs de k
  kmeans_model <- kmeans(velib_PCA_reduced, centers = k)
  Iintra=c(Iintra,kmeans_model$tot.withinss)
  
}
inertie_df=data.frame(K=2:10,Iintra=Iintra)
ggplot(inertie_df,aes(x=K,y=Iintra))+geom_line()+geom_point()+ 
  labs(x = "Nombre de clusters (k)", y = "Inertie") +
  ggtitle("Dispersion de l'inertie en fonction de k")

Un bon clustering minimise l'inertie intra-classes et maximise l'inter-classes. 
Pour trouver le nombre optimal de clusters, on fait la méthode du coude : on garde 4 clusters.

### Méthode silhouette et WSS

In [ ]:
g1=fviz_nbclust(velib_PCA_reduced, stats::kmeans, method = "silhouette")
g2=fviz_nbclust(velib_PCA_reduced, stats::kmeans, method = "wss")
grid.arrange(g1,g2)

Pour silhouette on doit prendre le max -> On choisit donc de garder 3 clusters selon cette méthode


### Graphes Silhouette

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 6)

#On teste pour plusieurs k différents

#k=2
kmeans_model <- kmeans(velib_PCA_reduced, centers=2, nstart=10)
silhouette_vals <- silhouette(kmeans_model$cluster, dist(velib_PCA_reduced))
p1=fviz_silhouette(silhouette_vals, main="Silhouette for k=2")

#k=3
kmeans_model <- kmeans(velib_PCA_reduced, centers=3, nstart=10)
silhouette_vals <- silhouette(kmeans_model$cluster, dist(velib_PCA_reduced))
p2=fviz_silhouette(silhouette_vals, main="Silhouette for k=3")

#k=4
kmeans_model <- kmeans(velib_PCA_reduced, centers=4, nstart=10)
silhouette_vals <- silhouette(kmeans_model$cluster, dist(velib_PCA_reduced))
p3=fviz_silhouette(silhouette_vals, main="Silhouette for k=4")

#k=5
kmeans_model <- kmeans(velib_PCA_reduced, centers=5, nstart=10)
silhouette_vals <- silhouette(kmeans_model$cluster, dist(velib_PCA_reduced))
p4=fviz_silhouette(silhouette_vals, main="Silhouette for k=5")

grid.arrange(p1,p2,p3, p4,ncol=2, nrow=2)

Ce graphe silouhette confirme les 3 clusters choisis à la méthode précédente: on a moins de valeurs négatives, les pics dépassent tous la moyenne (ligne rouge), qui est d'ailleurs la plus haute. On voit donc que 3 clusters est le meilleur choix pour Silhouette. 


Les méthodes Silouhette et inertie Intra ne donnent pas le même nombre de clusters. 
Nous choisissons arbitrairement 4 clusters pour la suite.

## Projection des individus selon leur cluster faits sur les 4 composantes principales

### Composantes 1 et 2

In [ ]:
#On garde donc 4 clusters
K = 4

palette = c("#8dd3c7", "#ffffb3", "#db564d", "#64ade8", "#fdb462", "#b3de69", "#fccde5", "#d9d9d9", "#bc80bd")

#Sur le jeu de données réduit
kmeans_pca <- kmeans(velib_PCA_reduced, centers = K, nstart = 10)

#On obtient les clusters prédits
clusters_pca <- kmeans_pca$cluster

In [ ]:
ggplot() + 
  geom_point(aes(x=velib_PCA_reduced[,1], y=velib_PCA_reduced[,2], col = as.factor(clusters_pca)))+
  scale_color_manual(values = palette) +
  labs(x = "1ère composante principale", y = "2nde composante principale", title = paste("Individuals factor map - Couleur selon les clusters k-means - Jeu réduit"))

### Composantes 1 et 3

In [ ]:
ggplot() + 
  geom_point(aes(x=velib_PCA_reduced[,1], y=velib_PCA_reduced[,3], col = as.factor(clusters_pca)))+
  scale_color_manual(values = palette) + 
  labs(x = "1ère composante principale", y = "3ème composante principale", title = paste("Individuals factor map - Couleur selon les clusters k-means - Jeu réduit"))

## Projection des individus selon leur cluster faits sur les données brutes

In [ ]:
kmeans_raw <- kmeans(loading, centers = K, nstart = 10)

#On obtient les clusters prédits
clusters_raw <- kmeans_raw$cluster

ggplot() + 
  geom_point(aes(x=velib_PCA_reduced[,1], y=velib_PCA_reduced[,2], col = as.factor(clusters_raw)))+
  scale_color_manual(values = palette) + 
  labs(x = "1ère composante principale", y = "2nde composante principale", title = paste("Individuals factor map - Couleur selon les clusters k-means - Jeu complet"))

On remarque que ces résultats entre les données brutes et les données réduites sont quasiment identiques.

Ceci montre que les 4 premières composantes expliquent la majorité des données, on a donc très peu de perte d'information en réduisant la dimension.

## Matrices de confusion pour comparer nos clusters

In [ ]:
#on calcule la matrice de confusion
cm <- table(clusters_pca, clusters_raw)
clusters_kmean_raw_sorted <- clusters_raw[order(clusters_pca)]

print(cm)
image(cm, main = "Matrice de confusion",
      xlab = "Avec l'algo kmeans PCA",
      ylab = "Avec l'algo kmeans sur les données complètes")

On trouve des grandes valeurs dans la diagonale, ce qui confirme les plots précédents : on a des résultats similaires sur le jeu de données complet et sur celui réduit (en gardant seulement les 4 composantes principales)


## Visualiser par cartographie le Kmeans sur le jeu de données réduit et complet:

### Jeu de données réduit

In [ ]:
df_clusters <- data.frame(
  latitude = Position[,2],
  longitude = Position[,1],
  cluster = as.factor(clusters_pca)
)

fig_cluster_pca <- plot_ly(data = df_clusters, x = ~longitude, y = ~latitude, type = "scattermapbox",
                           color = ~cluster, colors = palette[1:K],
                           marker = list(size = 7), zoom = 10000, text = ~cluster,
                           hoverinfo = "text") %>%
  layout(mapbox = list(center = list(lon = paris_lon, lat = paris_lat),
                                 zoom = 10, style = "carto-positron",title = "Individual factor map with clusters kmeans on PCA data"))

fig_cluster_pca

### Jeu de données complet

In [ ]:
df_clusters <- data.frame(
  latitude = Position[,2],
  longitude = Position[,1],
  cluster = as.factor(clusters_raw)
)

fig_cluster_raw <- plot_ly(data = df_clusters, x = ~longitude, y = ~latitude, type = "scattermapbox",
                           color = ~cluster, colors = palette[1:K],
                           marker = list(size = 7), zoom = 10, text = ~cluster,
                           hoverinfo = "text") %>%
                   layout(mapbox = list(center = list(lon = paris_lon, lat = paris_lat),
                                 zoom = 10, style = "carto-positron"))

fig_cluster_raw

On remarque qu'il n'y a presque aucun changement entre les deux cartographies, que ce soit pour les clusters faits sur les données PCA, ou les clusters faits sur les données brutes. Ceci nous donne le même résultat qu'obtenu par la table de contingence. 


## Que représentent ces 4 clusters Kmeans sur les données réduites ?

In [ ]:
for (i in 1:K) {
  indice <- which(clusters_pca == i)
  data_subset <- loading[indice, ]
  bp <- boxplot(data_subset, 
                col = (palette[1:K])[i], border = "black", median.col = "red",
                staplewex = 0, notch = FALSE, outline = FALSE,
                names = rep("", ncol(data_subset)))
  
  abline(v = seq(1, ncol(loading), length.out = 8), col = "purple", lty = "dotted")
  
  
  title <- paste("Boxplot pour le cluster", i)
  ticks <- seq(0, 168, by = 5)
  title(main = title, cex.main = 1.25)
  axis(1, at = ticks, labels = labels, cex.axis = 1.25)
  labels <- seq(0, 168, by = 5)
  mtext("Time", side = 1, line = 2, cex = 1.5)
  mtext("Loading", side = 2, line = 2, cex = 1.5)
}

On obtient des différences significatives pour les 4 clusters : 
- Cluster n°1 : Vide le soir et le matin, plein dans l'après-midi.
- Cluster n°2 : Inverse du n°1 : vide l'après-midi, plein le matin et le soir.
- Cluster n°3 : Souvent vide en général, quasiment tout le temps vide de 8h à 18h.
- Cluster n°4 : Souvent plein en général, quasiment tout le temps plein de 21h à 8h.

Rq : La numérotation des clusters peut changer en fonction de la compilation.

# Agglomerative Clustering

## Choix optimal du nombre de clusters

In [ ]:
#On choisit Ward pour la matrice de dissimilarités 

ac <- hclust(dist(velib_PCA_reduced), method = "ward.D2")

#Nombre optimal de clusters avec WSS et Silhouette
g1=fviz_nbclust(velib_PCA_reduced, FUN = hcut, method = "wss") + theme_minimal()
g2=fviz_nbclust(velib_PCA_reduced, FUN = hcut, method = "silhouette") + theme_minimal()

grid.arrange(g1,g2)

La méthode du coude pour WSS nous suggere de prendre 4 clusters.
Pour Silhouette on prendrait plutôt 3 clusters comme avec Kmeans.

De même que pour Kmeans, on décide de garder arbitrairement 4 clusters.

## Dendogramme coloré en fonction des classes

In [ ]:
#On prend 4 clusters
K <- 4

ac <- hclust(dist(velib_PCA_reduced), method = "ward.D2")

#On trace un dendrogramme
fviz_dend(ac,k=K)

# title("Dendrogram with Ward linkage") # ne marche pas

On voit bien les 4 clusters, chaque couleur correspond à un cluster.

## Projection des individus selon leur cluster faits sur les 4 composantes principales

In [ ]:
palette = c("#8dd3c7", "#ffffb3", "#db564d", "#64ade8", "#fdb462", "#b3de69", "#fccde5", "#d9d9d9", "#bc80bd")


ac <- hclust(dist(velib_PCA_reduced), method = "ward.D2")
#Découpage de l'arbre en K clusters
clusters_ac <- cutree(ac, k = K)

#Données PCA
g2=ggplot(velib_PCA_reduced, aes(x = velib_PCA_reduced[,1], y = velib_PCA_reduced[,2], color = as.factor(clusters_ac))) +
  geom_point(size = 1) +
  scale_color_manual(values = palette) +
  labs(title = "Scatter plot with clusters : données PCA") +
  theme_minimal()
g2

## Projection des individus selon leur cluster faits sur les données brutes

In [ ]:
#On fait sur loading
palette = c("#8dd3c7", "#ffffb3", "#db564d", "#64ade8", "#fdb462", "#b3de69", "#fccde5", "#d9d9d9", "#bc80bd")

# Clustering sur l'ensemble des données
ac <- hclust(dist(loading), method = "ward.D2")
clusters_ac_raw <- cutree(ac, k = K)

#Données complètes
g2=ggplot(velib_PCA_reduced, aes(x = velib_PCA_reduced[,1], y = velib_PCA_reduced[,2], color = as.factor(clusters_ac_raw))) +
  geom_point(size = 1) +
  scale_color_manual(values = palette) +
  labs(title = "Scatter plot with clusters : données complètes") +
  theme_minimal()
g2

On voit encore une fois que les deux clustering sont similaires. On ne pert donc pas trop d'information en faisant une PCA.


## Matrice de Confusion pour les clusters CAH sur le jeu de données réduit et brutes.

In [ ]:
#on calcule la matrice de confusion
cm <- table(clusters_ac, clusters_ac_raw)
clusters_ac_raw_sorted <- clusters_ac_raw[order(clusters_ac)]

print(cm)
image(cm, main = "Matrice de confusion",
      xlab = "Avec l'algo CAH PCA",
      ylab = "Avec l'algo CAH sur les données complètes")

On a des résultats similaires sur le jeu de données complet et sur celui réduit (en gardant seulement les 4 composantes principales), ce qui confirme les plots précédents.


## Visualiser par cartographie le CAH sur le jeu de données réduit et complet:

In [ ]:
df_clusters_pca <- data.frame(
  latitude = Position[,2],
  longitude = Position[,1],
  cluster = as.factor(clusters_ac)
)

fig_cluster_pca2 <- plot_ly(data = df_clusters_pca, x = ~longitude, y = ~latitude, type = "scattermapbox",
                    color = ~cluster, colors = palette[1:K],
                    marker = list(size = 7), zoom = 10, text = paste("Loading:", round(data24h, 2)),
                    hoverinfo = "text") %>%
            layout(mapbox = list(center = list(lon = paris_lon, lat = paris_lat),
                                 zoom = 10, style = "carto-positron",title = "Individual factor map with clusters CAH on PCA data"))



fig_cluster_pca2 <- fig_cluster_pca2 %>% layout(showlegend = TRUE)
fig_cluster_pca2

In [ ]:
#Fonction pour faire correspondre les classes
matchClasses <- function(classif1, classif2) {
  cm <- table(classif1, classif2)
  K <- nrow(cm)
  a <- numeric(K)
  b <- numeric(K)
  
  for (j in 1:K) {
    for (i in 1:K) {
      if (a[j] < cm[i, j]) {
        a[j] <- cm[i, j]
        b[j] <- i
      }
    }
  }
  
  table <- cm
  for (i in 1:K) {
    table[, b[i]] <- cm[, i]
  }
  
  clusters <- classif2
  n <- length(classif2)
  for (i in 1:n) {
    for (j in 1:K) {
      if (classif2[i] == j) {
        clusters[i] <- b[j]
      }
    }
  }
  
  return(list(table, clusters))
}

In [ ]:
result <- matchClasses(clusters_ac, clusters_ac_raw)
clusters_ac_raw_sorted <- result[[2]]

df_clusters_pca <- data.frame(
  latitude = Position[,2],
  longitude = Position[,1],
  cluster = as.factor(clusters_ac_raw_sorted)
)

fig_cluster_ac_raw <- plot_ly(data = df_clusters_pca, x = ~longitude, y = ~latitude, type = "scattermapbox",
                    color = ~cluster, colors = palette[1:K],
                    marker = list(size = 7), zoom = 10, text = paste("Loading:", round(data24h, 2)),
                    hoverinfo = "text") %>%
            layout(mapbox = list(center = list(lon = paris_lon, lat = paris_lat),
                                 zoom = 10, style = "carto-positron",title = "Individual factor map with clusters CAH on PCA data"))



fig_cluster_ac_raw <- fig_cluster_ac_raw %>% layout(showlegend = TRUE)
fig_cluster_ac_raw

De même, on observe peu de différences dans l'ensemble, sauf en périphérie !

## Comparaison clustering CAH et Kmeans sur données PCA

In [ ]:
#on calcule la matrice de confusion
cm <- table(clusters_ac, clusters_pca)
print(cm)
image(cm, main = "Matrice de confusion",
      xlab = "Avec l'algo CAH PCA",
      ylab = "Avec l'algo Kmeans PCA")

adjustedRandIndex(clusters_ac, clusters_pca)

On remarque une forte correspondance entre les 2 méthodes de clustering. Kmeans et CAH mènent donc à des clusters similaires.
Ceci est confimé par le ARI qui vaut 0.6 montrant une similarité entre ces deux clusterings.

# Gaussian Mixture Models

In [ ]:
gmm_model <- Mclust(velib_PCA_reduced, G = K)

## Selection du modèle et du nombre de clusters : Avec BIC

In [ ]:
resBICall = mclustBIC(velib_PCA_reduced, G=2:12)
summary(resBICall)

resBICall

resBICall = Mclust(velib_PCA_reduced, G=2:12)
summary(resBICall)

fviz_mclust(resBICall, what="BIC")

Le critère BIC sélectionne le modèle VVE (forme de la covariance : volumes différents, orientations égales et formes différentes)
On garde aussi 12 classes (en regardant le max du BIC)


# Projection des 12 clusters sur les 2 premières composantes principales

In [ ]:
options(repr.plot.width = 13, repr.plot.height = 6)

K = length(resBICall$parameters$mean[,][1,])
model_name = resBICall$parameters$variance$modelName
palette = c("#8dd3c7", "#ffffb3", "#bebada", "#db564d", "#64ade8", "#fdb462", "#b3de69", "#fccde5", "#d9d9d9", "#bc80bd", "#589336", "#ae3f59")

resBIC = Mclust(velib_PCA_reduced, G=K, modelNames = model_name)
fviz_cluster(resBIC, data=velib_PCA_reduced, ellipse.type="norm", geom="point")

## Boxplot des probabilités d'appartenance à une classe

aux = data.frame(
    label = paste("Cluster", resBIC$classification, sep=""), 
    proba = apply(resBIC$z, 1, max))

p1 = ggplot(aux, aes(x=label, y=proba)) + 
    geom_boxplot(colour=palette, fill=palette, alpha=.2)
p2 = fviz_cluster(resBIC, data =velib_PCA_reduced, ellipse.type="norm", geom="point") +
    ggtitle("") + theme(legend.position = "none")

grid.arrange(p1, p2, ncol = 2)

On observe pour chaque cluster le boxplot des probas d'appartenance de chaque individu. Par exemple le cluster 5 est le mieux défini : la proba moyenne des individus est proche de 1 et la variance est faible, les individus sont donc tous quasiment certains d'appartenir à ce cluster, il y a peu de mal classés dans ce dernier.

Au contraire, le cluster 9 a une grande variance est une moyenne de proba d'appartenance égale à 0.75, les individus sont donc moins certains d'appartenir à ce cluster.

## Projection des clusters sur la carte de Paris

In [ ]:
clusters_gmm <- resBIC$classification

df_clusters_gmm <- data.frame(
  latitude = Position[,2],
  longitude = Position[,1],
  cluster = as.factor(clusters_gmm)
)

fig_cluster_gmm <- plot_ly(data = df_clusters_gmm, x = ~longitude, y = ~latitude, type = "scattermapbox",
                    color = ~cluster, colors = palette,
                    marker = list(size = 7), zoom = 10, text = paste("Loading:", round(data24h, 2)),
                    hoverinfo = "text") %>%
            layout(mapbox = list(center = list(lon = paris_lon, lat = paris_lat),
                                 zoom = 10, style = "carto-positron",title = "Individual factor map with clusters GMM on PCA data"))



fig_cluster_gmm <- fig_cluster_gmm %>% layout(showlegend = TRUE)
fig_cluster_gmm

12 Clusters semble bien trop. On a de nombreux clusters qui ont une variance énorme, et qui ne semblent rien expliquer. Tentons un autre critère de sélection.

## Selection du modèle et du nombre de clusters : Avec ICL

In [ ]:
resICLall_1 = mclustICL(velib_PCA_reduced, G=2:12, verbose = FALSE)
summary(resICLall_1)

Avec le critère ICL, on garde également le modèle VVE mais avec seulement 6 clusters.

In [ ]:
plot(resICLall_1)

On retrouve bien le max d'ICL pour le modèle précédent.

# Projection des 6 clusters sur les 2 premières composantes principales

In [ ]:
options(repr.plot.width = 13, repr.plot.height = 6)

K = 6
model_name = "VVE"

resICL = Mclust(velib_PCA_reduced, G=K, modelNames=model_name, verbose = FALSE)
fviz_cluster(resICL, data=velib_PCA_reduced, ellipse.type="norm", geom="point")

## Boxplot des probabilités d'appartenance à une classe

In [ ]:
aux = data.frame(
    label = paste("Cl", resICL$classification, sep=""), 
    proba = apply(resICL$z, 1, max))

palette = palette[1:K]

p1 = ggplot(aux, aes(x=label, y=proba)) + 
    geom_boxplot(colour=palette, fill=palette, alpha=.2)
p2 = fviz_cluster(resBIC, data =velib_PCA_reduced, ellipse.type="norm", geom="point") +
    ggtitle("") + theme(legend.position = "none")

grid.arrange(p1, p2, ncol = 2)

Ici on voit que les clusters 2,3 et 5 sont ceux où les individus ont une plus grande probabilité d'appartenance. On remarque en général que tous les clusters sont bien définis car les moyennes sont hautes.

## Projection des 6 clusters sur la carte de Paris

In [ ]:
clusters_gmm_icl <- resICL$classification

df_clusters_gmm_icl <- data.frame(
  latitude = Position[,2],
  longitude = Position[,1],
  cluster = as.factor(clusters_gmm_icl)
)

fig_cluster_gmm_icl <- plot_ly(data = df_clusters_gmm_icl, x = ~longitude, y = ~latitude, type = "scattermapbox",
                    color = ~cluster, colors = palette,
                    marker = list(size = 7), zoom = 10, text = paste("Loading:", round(data24h, 2)),
                    hoverinfo = "text") %>%
            layout(mapbox = list(center = list(lon = paris_lon, lat = paris_lat),
                                 zoom = 10, style = "carto-positron",title = "Individual factor map with clusters GMM on PCA data"))



fig_cluster_gmm_icl <- fig_cluster_gmm_icl %>% layout(showlegend = TRUE)
fig_cluster_gmm_icl

## Interprétation des 6 clusters

In [ ]:
for (i in 1:K) {
  indice <- which(clusters_gmm_icl == i)
  data_subset <- loading[indice, ]
  bp <- boxplot(data_subset, 
                col = palette[i], border = "black", median.col = "red",
                staplewex = 0, notch = FALSE, outline = FALSE,
                names = rep("", ncol(data_subset)))
  
  abline(v = seq(1, ncol(loading), length.out = 8), col = "purple", lty = "dotted")
  
  
  title <- paste("Boxplot pour le cluster", i)
  ticks <- seq(0, 168, by = 5)
  labels <- seq(0, 168, by = 5)
  title(main = title, cex.main = 1.25)
  axis(1, at = ticks, labels = labels, cex.axis = 1.25)
  mtext("Time", side = 1, line = 2, cex = 1.5)
  mtext("Loading", side = 2, line = 2, cex = 1.5)
} # faire en sorte que ordonnée fixe

Cluster 1 : quartiers rédientiels (classe moyenne?) très déchargés de 8h à 18h(~0 de loading), un peu plus rechargés de 18h à 8h (~0.5 de loading) <br>
Cluster 2 : le long de la seine, tourisme + études + gare : souvent chargés la journée (~0.9 de loading), plus déchargés en soirée et la nuit (~ 0.2 de loading)<br>
Cluster 3 : bizarre, pas de tendance globale du tout. <br>
Cluster 4 : on retrouve beaucoup de points d'intérêt, lieux de divertissements (tour eiffel, bercy, bastille) -> les gens s'y rendent plus qu'à d'autres endroits, ils y posent donc leur vélo en plus grand nombre : les stations plus souvent chargées. <br>
Cluster 5 : Zones similaires au cluster 6, Stations délaissées : peu de variance, pas de chargement au dessus de 0.4. <br>
Cluster 6 : champs élysées, opéra (zones d'intérêt, commerciales) : très chargé la journée, très déchargé le soir/nuit et le week-end -> surutilisé. <br>

On peut trouver une bonne explication pour tous les clusters sauf le 3; beaucoup de variance, loading quasi tout le temps à 0.5... pas de comportement distinctif. On ne peut pas extraire d'information du cluster 3. De plus, les cluster 6 et 7 se ressemblent beaucoup. On va donc essayer de ""supprimer"" un cluster.

## GMM avec 5 clusters :

In [ ]:
options(repr.plot.width = 13, repr.plot.height = 6)

K = 5
model_name = "VVE" # VVE semble être le meilleur paramètre selon BIC et ICL. 

resICL = Mclust(velib_PCA_reduced, G=K, modelNames=model_name, verbose = FALSE)
fviz_cluster(resICL, data=velib_PCA_reduced, ellipse.type="norm", geom="point")

# --- #

aux = data.frame(
    label = paste("Cl", resICL$classification, sep=""), 
    proba = apply(resICL$z, 1, max))

palette = palette[1:K]

p1 = ggplot(aux, aes(x=label, y=proba)) + 
    geom_boxplot(colour=palette, fill=palette, alpha=.4)
p2 = fviz_cluster(resICL, data=velib_PCA_reduced, ellipse.type="norm", geom="point") +
    ggtitle("") + theme(legend.position = "none")

grid.arrange(p1, p2, ncol = 2)

Ici on voit que le cluster n°3 est celui où les individus ont une plus grande probabilité d'appartenance. On remarque en général que tous les clusters sont bien définis car les moyennes sont hautes (on a par contre des variances plus grandes pour les autres clusters que le 3)

## Projection des 5 clusters sur la carte de Paris

In [ ]:
clusters_gmm_icl <- resICL$classification

df_clusters_gmm_icl <- data.frame(
  latitude = Position[,2],
  longitude = Position[,1],
  cluster = as.factor(clusters_gmm_icl)
)

fig_cluster_gmm_icl <- plot_ly(data = df_clusters_gmm_icl, x = ~longitude, y = ~latitude, type = "scattermapbox",
                    color = ~cluster, colors = palette,
                    marker = list(size = 7), zoom = 10, text = paste("Loading:", round(data24h, 2)),
                    hoverinfo = "text") %>%
            layout(mapbox = list(center = list(lon = paris_lon, lat = paris_lat),
                                 zoom = 10, style = "carto-positron",title = "Individual factor map with clusters GMM on PCA data"))



fig_cluster_gmm_icl <- fig_cluster_gmm_icl %>% layout(showlegend = TRUE)
fig_cluster_gmm_icl

## Interprétation des 5 clusters

In [ ]:
for (i in 1:K) {
  indice <- which(clusters_gmm_icl == i)
  data_subset <- loading[indice, ]
  bp <- boxplot(data_subset, 
                col = palette[i], border = NULL, median.col = "red",
                staplewex = 0, notch = FALSE, outline = FALSE,
                names = rep("", ncol(data_subset)))
  
  abline(v = seq(1, ncol(loading), length.out = 8), col = "purple", lty = "dotted")
  
  
  title <- paste("Boxplot pour le cluster", i)
  ticks <- seq(0, 168, by = 5)
  labels <- seq(0, 168, by = 5)
  title(main = title, cex.main = 1.25)
  axis(1, at = ticks, labels = labels, cex.axis = 1.25)
  mtext("Time", side = 1, line = 2, cex = 1.5)
  mtext("Loading", side = 2, line = 2, cex = 1.5)
} # faire en sorte que ordonnée fixe

Cluster 1 : Vide en journée (8-18h, ~0 loading), plus rempli la nuit (~0.7 loading), et peu rempli le week-end
Cluster 2 : Globalement plûtot rempli (~0.4/0.6), mais pas de comportement distinctif : trop de variance
Cluster 3 : Plein en journée (8-18h, ~0.90 loading), plus vide la nuit (~0.1 loading). Tendance similaire le week-end.
Cluster 4 : Plutôt rempli en journée (8-18h, ~0.90 loading), plus vide la nuit et le week-end (~0.1 loading).
Cluster 5 : Globalement peu rempli. (~0.1 loading) Comportement "chaotique".

Note : L'ordre des clusters est sujet à changer, mais l'interprétation de ces derniers reste la même. Ainsi, on obtient 4 clusters très cohérents, et remplis d'informations distinctives, tandis qu'un cluster reste moins intéressant.

## Comparaison clustering GMM et Kmeans

In [ ]:
#on calcule la matrice de confusion
cm <- table(clusters_gmm_icl, clusters_pca)
print(cm)
image(cm, main = "Matrice de confusion",
      xlab = "Avec l'algo GMM sur PCA",
      ylab = "Avec l'algo Kmeans PCA")

adjustedRandIndex(clusters_gmm_icl, clusters_pca)

L'index ajusté vaut 0.34 ce qui montre que les deux clusterings ne correspondent pas beaucoup.

## Comparaison clustering GMM et CAH

In [ ]:
#on calcule la matrice de confusion
cm <- table(clusters_gmm_icl, clusters_ac)
print(cm)
image(cm, main = "Matrice de confusion",
      xlab = "Avec l'algo GMM sur PCA",
      ylab = "Avec l'algo CAH PCA")
adjustedRandIndex(clusters_gmm_icl, clusters_ac)

L'index ajusté vaut 0.32 ce qui montre que les deux clusterings correspondent encore moins que pour GMM et Kmeans

## MCA

Après avoir fait la PCA pour réduire les dimensions du jeu de données, on essyae de faire une MCA, c'est à dire une Analyse des Correspondances Multiples, sur le jeu de donnée entier.

Comme pour la CA, la MCA fonctionne sur des variables qualitatives, donc on doit transformer les données en qualitatives avec des seuils ! Contrairement à la CA que l'on effectuait sur des matrices de confusions donc des données déjà qualitatives.

## MCA sur toutes les données

In [ ]:
seuils = list(
    'Vide'= 0.2,
    'Faible'= 0.4,
    'Moyen'= 0.6,
    'Fort'= 0.8,
    'Pleine'= 1.0)

seuilsV2 = list(
    'Vide' = 0.33,
    'Moyen' = 0.66,
    'Pleine' = 1.0)

seuilsV3 = list(
    'Totalement Vide'= 0.1,
    'Très Vide' = 0.2,
    'Assez Faible' = 0.3,
    'Faible' = 0.4,
    'Moyen' = 0.5,
    'Pas mal' = 0.6,
    'Bien' = 0.7,
    'Très Bien' = 0.8,
    'Quasiment Pleine' = 0.9,
    'Pleine' = 1.0
)

# Définis les seuils
seuilsV4 = list(
    'Vide'= 0.5,
    'Plein'= 1
)

loading_quali <- data.frame(sapply(loading, function(col) cut(col, breaks = c(-Inf, unlist(seuils), Inf), labels = c(names(seuils), 'Pleine'))))
loading_quali$hill <- data.frame(hill = Coord$bonus)
loading_quali <- loading_quali %>% mutate(hill = case_when(loading_quali$hill==0 ~ "Nohill",
                                         loading_quali$hill==1 ~ "Hill"))

On décide d'utiliser 5 seuils, on utilisera plus tard 3 seuils, car sur ce jeu de données on a beaucoup de variables qualitatives donc 3 modalités par variables pourrait être intéressant, mais nous voulions quand même essayer pour 5 seuils et voir à quel point la MCA est interprétable.

In [ ]:
res.mca <- MCA(loading_quali, graph = TRUE)

head(res.mca$eig, 24)

In [ ]:
fviz_mca_var(res.mca, repel=TRUE)

#On affiche les variables et leur modalités dans le plan des composantes 0 et 1 de la MCA

Le graphe ci-dessous est peu exploitable, car beaucoup trop encombré.

In [ ]:
eig <- data.frame(res.mca$eig)

tick = 1:length(eig$cumulative.percentage.of.variance)

#Variance expliquée par composante principale
barplot(height = eig$percentage.of.variance, names.arg = tick,
        main = "Variance expliquée par composante principale",
        xlab = "Composante principale",
        ylab = "Variance expliquée")

barplot(height = eig$cumulative.percentage.of.variance, names.arg = tick,
        main = "Variance expliquée cumulée par composante principale de la MCA",
        xlab = "Composante principale",
        ylab = "Variance expliquée cumulée")
abline(h = 70, col="red")

On remarque qu'il nous faudrait garder à peu près 100 composantes pour expliquer 70% de la variance. Ainsi, on se rend compte que la MCA sur le jeu de données entier n'est pas du tout efficace en terme d'explication de variances, ce qui nous laisse penser que l'on devrait refaire une MCA sur un jeu de données légèrement modifié.

On continue cependant légèrement l'analyse de cette MCA avant de modifier notre jeu de données !

In [ ]:
fviz_mca_ind(res.mca,
label = "none", # hide individual labels
habillage = "hill", # color by groups
palette = c("blue", "#87e364", "#eb5656", "purple", "gold"),
axes = c(1,2)
#addEllipses = TRUE, ellipse.type = "confidence",
)

fviz_mca_ind(res.mca,
label = "none", # hide individual labels
habillage = "Lun.00", # color by groups
palette = c("blue", "#87e364", "#eb5656", "purple", "gold"),
axes = c(1,2)
#addEllipses = TRUE, ellipse.type = "confidence",
)

fviz_mca_ind(res.mca,
label = "none", # hide individual labels
habillage = "hill", # color by groups
palette = c("blue", "#87e364", "#eb5656", "purple", "gold"),
axes = c(2,3)
#addEllipses = TRUE, ellipse.type = "confidence",
)

fviz_mca_ind(res.mca,
label = "none", # hide individual labels
habillage = "Lun.00", # color by groups
palette = c("blue", "#87e364", "#eb5656", "purple", "gold"),
axes = c(2,3)
#addEllipses = TRUE, ellipse.type = "confidence",
)

On essaye désormais de voir la contributions des colonnes, donc des variables qualitatives et de leur modalité, par les composantes de cette MCA :

In [ ]:
contrib = res.mca$var$contrib
head(contrib, 9)

COMPLÉTER

## MCA sur les données jours nuit

Après avoir tester de faire une MCA sur le jeu de données dans son entierté nous décidons donc de regrouper les données. Comme nous remarquons depuis l'naalyse exploratoire que des séparations se faisaient au niveau du Jour/Nuit *(notamment via les corrplots)* et du Semaine/Week-end *(via le chargement moyen en fonction des jours et des heures notamment)* nous avons décider de séparer les données en journée et en nuit pour bien marquer cette différence tout en gardant les jours de la semaine et du week-end bien différentiable !

Ainsi notre répartition est après analyse des corrplots de l'analyse descriptive : 

- **Jour : De 09h à 19h**

- **Nuit : De 20h à 08h le lendemain** 

*(Exemple : Lundi nuit représente donc la période du Lundi 20h au Mardi 8h)*

- **Et le week-end se tient donc de Vendredi nuit à Dimanche nuit compris !** Donc de Nuit 5 à Nuit 7 compris

In [ ]:
X_quali <- data.frame(hill = Coord$bonus)
X_quali <- X_quali %>% mutate(hill = case_when(X_quali$hill==0 ~ "Nohill",
                                         X_quali$hill==1 ~ "Hill"))
head(X_quali, n=11)

In [ ]:
load = c('vide', 'moyen', 'chargé')
for (i in 1:7){
i_debut_jour = (i-1)*24 + 1 + 9
i_fin_jour = i*24 - 4
new_Col = rowMeans(loading[,i_debut_jour:i_fin_jour])
print(loading[,i_debut_jour:i_fin_jour])
breaks = c(-0.1,0.33, 0.66, 1.01)
newCol = cut(new_Col, breaks = breaks, labels = load)
colname = paste("Jour", i, sep="")
X_quali[[colname]] = newCol
}
head(newCol)
head(X_quali)

In [ ]:
len_X = 24 * 7

for (i in 1:7){
  i_debut_nuit = i*24 - 3
  if (i*24!=len_X){
    i_fin_nuit = i*24 + 9
    new_Col = rowMeans(loading[,i_debut_nuit:i_fin_nuit])
  } else{
    i_fin_nuit = (i*24 + 9) %% len_X
    nuit_dimanche = group_by(loading[i_debut_nuit:len_X],loading[1:i_fin_nuit])
    new_Col = rowMeans(nuit_dimanche)
  }
  breaks = c(-0.1,0.33, 0.66, 1.01)
  newCol = cut(new_Col, breaks = breaks, labels = load)
  colname = paste("Nuit", i, sep="")
  X_quali[[colname]] = newCol
  }

head(newCol)
head(X_quali)

In [ ]:
res.mca <- MCA(X_quali, graph = TRUE)

In [ ]:
eig <- data.frame(res.mca$eig)

tick = 1:length(eig$cumulative.percentage.of.variance)

#Variance expliquée par composante principale
barplot(height = eig$percentage.of.variance, names.arg = tick,
        main = "Variance expliquée par composante principale",
        xlab = "Composante principale",
        ylab = "Variance expliquée")

barplot(height = eig$cumulative.percentage.of.variance, names.arg = tick,
        main = "Variance expliquée cumulée par composante principale de la MCA",
        xlab = "Composante principale",
        ylab = "Variance expliquée cumulée")
abline(h = 70, col="red")

Pour 3 seuils à "0.33", "0.66" et "1" on a besoin de 10 composantes pour expliquer 70% de la variance ainsi on a beaucoup plus de composantes qu'en PCA, ce qui rend le jeu de données MCA plus complexe à manipuler si l'on veut conserver une bonne explication de la variance

In [ ]:
fviz_mca_ind(res.mca,
label = "none", # hide individual labels
habillage = "Jour2", # color by groups
palette = c("#87e364", "#ffd338", "#eb5656"),
axes = c(1,2)
#addEllipses = TRUE, ellipse.type = "confidence",
)
fviz_mca_ind(res.mca,
label = "none", # hide individual labels
habillage = "Jour2", # color by groups
palette = c("#87e364", "#ffd338", "#eb5656"),
axes = c(1, 3)
#addEllipses = TRUE, ellipse.type = "confidence",
)


fviz_mca_ind(res.mca,
label = "none", # hide individual labels
habillage = "hill", # color by groups
palette = c("steelblue", "orange"),
axes = c(1,2)
)
fviz_mca_ind(res.mca,
label = "none", # hide individual labels
habillage = "hill", # color by groups
palette = c("steelblue", "orange"),
axes = c(1,3)
)

On remarque que les stations Hill ont l'air d'être assez regroupées sur les dimensions ci-dessus, de mêmes que pour les modalités faible et forte de la journée du Mardi, ce qui est assez intéressant quand on compare cela à la MCA sur le jeu de données pas transformés !

In [ ]:
C <- as.data.frame(res.mca$ind$coord)
velib_MCA_reduced <- C

In [ ]:
contrib = res.mca$var$contrib
head(contrib, 44)

Visualisation des contributions, ainsi que de la qualité de représentation dans le plan MCA

In [ ]:
fviz_mca_var(res.mca, col.var = "contrib", repel = TRUE, gradient.cols = c("#00afbb","red","yellow"))

In [ ]:
fviz_mca_var(res.mca, col.var = "cos2", repel = TRUE, gradient.cols = c("#00afbb","red","yellow"))

In [ ]:
fviz_mca_ind(res.mca,
label = "none",
col.ind = "contrib",
axes = c(1,2))

fviz_mca_ind(res.mca,
label = "none",
col.ind = "cos2",
axes = c(1,2))

fviz_mca_ind(res.mca,
label = "none",
col.ind = "contrib",
axes = c(1,3))

fviz_mca_ind(res.mca,
label = "none",
col.ind = "cos2",
axes = c(1,3))

Après analyse de la MCA, on remarque donc qu'elle peut être assez pertinente pour réduire les dimensions de notre jeu de données Vélib, mais on préfèrerait quand même garder la PCA car on n'y utilise que 4 dimensions, la ou la MCA nous en fait utiliser 10.

### Clustering Kmeans sur les données MCA

In [ ]:
K=10
Iintra=NULL
for (k in 2:K)
{
  #On calcule l'inertie pour différentes valeurs de k
  kmeans_model <- kmeans(velib_MCA_reduced, centers = k)
  Iintra=c(Iintra,kmeans_model$tot.withinss)
  
}
inertie_df=data.frame(K=2:10,Iintra=Iintra)
ggplot(inertie_df,aes(x=K,y=Iintra))+geom_line()+geom_point()+ 
  labs(x = "Nombre de clusters (k)", y = "Inertie") +
  ggtitle("Dispersion de l'inertie en fonction de k")

# On garderait 3 clusters d'après la méthode du coude

In [ ]:
g1=fviz_nbclust(velib_MCA_reduced, stats::kmeans, method = "silhouette")
g2=fviz_nbclust(velib_MCA_reduced, stats::kmeans, method = "wss")
grid.arrange(g1,g2)

# La méthode silhouette propose 9 clusters mais cela paraît trop, et à partir de 2 clusters le score silhouette est quasi constant.

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 6)

#On teste pour plusieurs k différents

#k=2
kmeans_model <- kmeans(velib_MCA_reduced, centers=2, nstart=10)
silhouette_vals <- silhouette(kmeans_model$cluster, dist(velib_MCA_reduced))
p1=fviz_silhouette(silhouette_vals, main="Silhouette for k=2")

#k=3
kmeans_model <- kmeans(velib_MCA_reduced, centers=3, nstart=10)
silhouette_vals <- silhouette(kmeans_model$cluster, dist(velib_MCA_reduced))
p2=fviz_silhouette(silhouette_vals, main="Silhouette for k=3")

#k=4
kmeans_model <- kmeans(velib_MCA_reduced, centers=4, nstart=10)
silhouette_vals <- silhouette(kmeans_model$cluster, dist(velib_MCA_reduced))
p3=fviz_silhouette(silhouette_vals, main="Silhouette for k=4")

#k=5
kmeans_model <- kmeans(velib_MCA_reduced, centers=5, nstart=10)
silhouette_vals <- silhouette(kmeans_model$cluster, dist(velib_MCA_reduced))
p4=fviz_silhouette(silhouette_vals, main="Silhouette for k=5")

grid.arrange(p1,p2,p3, p4,ncol=2, nrow=2)

# Comme vu précédemment, le score silhouette n'invalide pas de clusters en dessous de 5. Prenons arbitrairement K = 5 clusters.

In [ ]:
#On garde donc 4 clusters
K = 5

palette = c("#8dd3c7", "#ffffb3", "#db564d", "#64ade8", "#fdb462", "#b3de69", "#fccde5", "#d9d9d9", "#bc80bd")

#Sur le jeu de données réduit
kmeans_mca <- kmeans(velib_MCA_reduced, centers = K, nstart = 10)

#On obtient les clusters prédits
clusters_mca <- kmeans_mca$cluster

In [ ]:
print(velib_MCA_reduced)

ggplot() + 
  geom_point(aes(x=velib_MCA_reduced[,1], y=velib_MCA_reduced[,2], col = as.factor(clusters_mca)))+
  scale_color_manual(values = palette) +
  labs(x = "1ère composante principale", y = "2nde composante principale", title = paste("Individuals factor map - Couleur selon les clusters k-means - Jeu réduit"))

In [ ]:
df_clusters <- data.frame(
  latitude = Position[,2],
  longitude = Position[,1],
  cluster = as.factor(clusters_mca)
)

fig_cluster_mca <- plot_ly(data = df_clusters, x = ~longitude, y = ~latitude, type = "scattermapbox",
                           color = ~cluster, colors = palette[1:K],
                           marker = list(size = 7), zoom = 10000, text = ~cluster,
                           hoverinfo = "text") %>%
  layout(mapbox = list(center = list(lon = paris_lon, lat = paris_lat),
                                 zoom = 10, style = "carto-positron",title = "Individual factor map with clusters kmeans on MCA data"))

fig_cluster_mca

In [ ]:
K = 5

palette = c("#8dd3c7", "#ffffb3", "#db564d", "#64ade8", "#fdb462", "#b3de69", "#fccde5", "#d9d9d9", "#bc80bd")

#Sur le jeu de données réduit
kmeans_raw <- kmeans(loading, centers = K, nstart = 10)

#On obtient les clusters prédits
clusters_raw <- kmeans_raw$cluster

In [ ]:
df_clusters <- data.frame(
  latitude = Position[,2],
  longitude = Position[,1],
  cluster = as.factor(clusters_raw)
)

fig_cluster_raw <- plot_ly(data = df_clusters, x = ~longitude, y = ~latitude, type = "scattermapbox",
                           color = ~cluster, colors = palette[1:K],
                           marker = list(size = 7), zoom = 10000, text = ~cluster,
                           hoverinfo = "text") %>%
  layout(mapbox = list(center = list(lon = paris_lon, lat = paris_lat),
                                 zoom = 10, style = "carto-positron",title = "Individual factor map with clusters kmeans on raw data"))

fig_cluster_raw

In [ ]:
##Interprétation des 5 clusters

for (i in 1:K){
  indice <- which(clusters_mca == i)
  data_subset <- loading[indice, ]
  bp <- boxplot(data_subset, 
                col = palette[i], border = NULL, median.col = "red",
                staplewex = 0, notch = FALSE, outline = FALSE,
                names = rep("", ncol(data_subset)))
  
  abline(v = seq(1, ncol(loading), length.out = 8), col = "purple", lty = "dotted")
  
  
  title <- paste("Boxplot pour le cluster", i)
  ticks <- seq(0, 168, by = 5)
  labels <- seq(0, 168, by = 5)
  title(main = title, cex.main = 1.25)
  axis(1, at = ticks, labels = labels, cex.axis = 1.25)
  mtext("Time", side = 1, line = 2, cex = 1.5)
  mtext("Loading", side = 2, line = 2, cex = 1.5)
} # faire en sorte que ordonnée fixe

On retrouve la même chose que précédemment pour le modèle GMM : On a 4 clusters bien ajustés, mais un cluster possède trop de variance, et n'est pas analysable.

## Comparaison des clusters sur le jeu de données MCA

### Avec le jeu de données complet

In [ ]:
#on calcule la matrice de confusion
cm_mca <- table(clusters_mca, clusters_raw)
clusters_kmean_raw_sorted <- clusters_raw[order(clusters_mca)]

print(cm_mca)
image(cm_mca, main = "Matrice de confusion",
      xlab = "Avec l'algo kmeans MCA",
      ylab = "Avec l'algo kmeans sur les données complètes")
adjustedRandIndex(clusters_mca, clusters_raw)

On remarque une bonne similarité car la matrice est quasiment diagonale, donc on pourrait dire que la MCA conserve bien les particularités jeu de données tout en réduisant les dimensions.
Ensuite on peut comparer le clustering MCA avec le clustering PCA voir s'ils sont similaire, si oui, on pourrait en déduire que le PCA est définitivement mieux a uvu de son nombre de dimensions inférieurs.

### Avec le jeu de données PCA

In [ ]:
K = 5

palette = c("#8dd3c7", "#ffffb3", "#db564d", "#64ade8", "#fdb462", "#b3de69", "#fccde5", "#d9d9d9", "#bc80bd")

#Sur le jeu de données réduit
kmeans_pca <- kmeans(velib_PCA_reduced, centers = K, nstart = 10)

#On obtient les clusters prédits
clusters_pca <- kmeans_pca$cluster

In [ ]:
cm_mca_pca <- table(clusters_mca, clusters_pca)
clusters_kmean_pca_sorted <- clusters_pca[order(clusters_mca)]

print(cm_mca_pca)
image(cm_mca_pca, main = "Matrice de confusion",
      xlab = "Avec l'algo kmeans MCA",     
      ylab = "Avec l'algo kmeans sur les données complètes")
adjustedRandIndex(clusters_mca, clusters_pca)

On remarque également une bonne similarité car la matrice est quasiment diagonale et à un ARI de presque 70%, donc on pourrait dire que la MCA et la PCA ont des clsuterings similaires avec kmeans, mais que la PCA réduit mieux les dimensions.